<a href="https://colab.research.google.com/github/catalinamagalvan/PySimMIBCI/blob/main/FBCNet%20Toolbox/Example_cross_validation_simdataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#10-fold cross-validation classification with simulated data
This Google colab notebook illustrates how to train and test FBCNet model with simulated data in a 10-fold cross-validation scenario (train: 8 folds, validation: 1 fold, test: 1 fold). In this example, simulated data comprise 200 simulated trials from subject S20 (%ERD: 20%, no failed trials). FBCNet was used as decoding model and one random seed is considered.


Notes:

*   Here, the simulated data was previously generated and is directly downloaded in order to reduce execution times. For more information on how to generate these MI-EEG data go to the example ["Example_generate_data_different_user_capabilities.ipynb"](https://github.com/catalinamagalvan/PySimMIBCI/blob/main/PySimMIBCI%20Toolbox/Example_generate_data_different_user_capabilities.ipynb).
*   FBCNet codes are a modification of the [original implementation](https://https://github.com/ravikiran-mane/FBCNet).

##Clone the GitHub repository

In [1]:
!git clone https://github.com/catalinamagalvan/PySimMIBCI.git

Cloning into 'PySimMIBCI'...
remote: Enumerating objects: 277, done.
remote: Counting objects: 100% (191/191), done.
remote: Compressing objects: 100% (127/127), done.
remote: Total 277 (delta 80), reused 109 (delta 41), pack-reused 86
Receiving objects: 100% (277/277), 1.16 MiB | 2.66 MiB/s, done.
Resolving deltas: 100% (122/122), done.


##Install and import required Python libraries

In [2]:
!pip install numpy==1.22.3
!pip install scipy==1.8.1
!pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0 --extra-index-url https://download.pytorch.org/whl/cu113

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 19.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 MB 18.1 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Uninstalling scipy-1.7.3:
      Successfully uninstalled scipy-1.7.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu113
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 1.6/1.6 GB 119.4 MB/s eta 0:00:01tcmalloc: large alloc 1636999168 bytes == 0x3b1c000 @  0x7f28684a4680 0x7f28684c5824 0x5b3128 0x5bbc90 0x5f714c 0x64d800 0x527022 0x504866 0x56bbe1 0x569d8a 0x5f60c3 0x56bbe1 0x569d8a 0x5f60c3 0x56bbe1 0x569d8a 0x5f60c3 0x56bbe1 0x569d8a 0x5f60c3 0x56bbe1 0x569d8a 0x5f60c3 0x56bbe1 0x5f5ee6 0x56bbe1 0x569d8a 0x5f60c3 0x56cc92 0x569d8a 0x5f60c3
tcmalloc: large alloc 2046255104 bytes == 0x65446000 @  0x7f28684a4680 0x7f28684c4da2 0x5f714c 0x64d800 0x52

In [3]:
import sys
import os

sys.path.insert(1, os.path.join('PySimMIBCI', 'FBCNet Toolbox', 'codes',
                                'centralRepo'))
from saveData import fetchData
from classification_scenarios import cross_validation

##Download and prepare simulated data

This step must be done in order to make sure that all needed data is present and to generate files with suitable format for FBCNet training.

In [4]:
if not os.path.exists(os.path.join('PySimMIBCI', 'FBCNet Toolbox', 'data',
                                   'simdataset', 'originalData')):
  os.makedirs(os.path.join('PySimMIBCI', 'FBCNet Toolbox', 'data', 'simdataset',
                           'originalData'))
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://drive.google.com/uc?export=download&id=1C1gVwUmyxY9bS6N3UegZpCjC8f8Koq-z' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1C1gVwUmyxY9bS6N3UegZpCjC8f8Koq-z" -O PySimMIBCI/FBCNet\ Toolbox/data/simdataset/originalData/s20.mat && rm -rf /tmp/cookies.txt

--2023-01-23 16:51:47--  https://docs.google.com/uc?export=download&confirm=t&id=1C1gVwUmyxY9bS6N3UegZpCjC8f8Koq-z
Resolving docs.google.com (docs.google.com)... 142.250.4.139, 142.250.4.100, 142.250.4.138, ...
Connecting to docs.google.com (docs.google.com)|142.250.4.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-04-6o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/5di41d702h98ta81rp29ab5v7hnsncqr/1674492675000/07130511430595189058/*/1C1gVwUmyxY9bS6N3UegZpCjC8f8Koq-z?e=download&uuid=cd9da330-eb25-4656-998f-1ba988e3a38c [following]
--2023-01-23 16:51:47--  https://doc-04-6o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/5di41d702h98ta81rp29ab5v7hnsncqr/1674492675000/07130511430595189058/*/1C1gVwUmyxY9bS6N3UegZpCjC8f8Koq-z?e=download&uuid=cd9da330-eb25-4656-998f-1ba988e3a38c
Resolving doc-04-6o-docs.googleusercontent.com (doc-04-6o-docs.googleusercontent.com)... 142.250.4.132, 2404:

In [5]:
fetchData('simdataset', subjects=['s20'])

Checking data for dataset:  simdataset
Extracting the data into mat format: 
Processing subject: s20
Converting to eegdataset.
Creating python eegdataset with raw data.
Converting to multi-view eegdataset.
Outputs will be saved in folder : /content/PySimMIBCI/FBCNet Toolbox/data/simdataset/multiviewPython
All the data you need is present! 


##Train and test the model

In [6]:
cross_validation(dataset='simdataset', network='FBCNet', random_seed_list=[0])

Se han truncado las últimas 5000 líneas del flujo de salida.
	 	 Epoch 267
Train loss = 0.737 Train Acc = 0.656 Val Acc = 0.700 Val loss = 0.871
	 	 Epoch 268
Train loss = 1.637 Train Acc = 0.544 Val Acc = 0.500 Val loss = 1.738
	 	 Epoch 269
Train loss = 0.407 Train Acc = 0.787 Val Acc = 0.700 Val loss = 0.728
	 	 Epoch 270
Train loss = 0.294 Train Acc = 0.875 Val Acc = 0.750 Val loss = 0.581
	 	 Epoch 271
Train loss = 0.297 Train Acc = 0.875 Val Acc = 0.750 Val loss = 0.587
	 	 Epoch 272
Train loss = 0.211 Train Acc = 0.944 Val Acc = 0.800 Val loss = 0.463
Exp No. : 1
________________________________________________

 Train Results: 
{'acc': 0.90625, 'cm': array([[69, 11],
       [ 4, 76]]), 'loss': 0.2295166254043579}

 Validation Results: 
{'acc': 0.85, 'cm': array([[8, 2],
       [1, 9]]), 'loss': 0.4133800506591797}

 Test Results: 
{'acc': 0.95, 'cm': array([[ 9,  1],
       [ 0, 10]]), 'loss': 0.22775840759277344}
Code will be running on device  cuda:0
Results will be saved in 